In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import statsmodels.api as sm
import warnings

In [28]:
# def plot(file):
#     df=pd.read_csv('./../data/composite_vars/' + file)
#     df['date'] = pd.to_datetime(df['date'])
#     df.set_index('date', inplace=True)
#     plt.figure(figsize=(14, 7))
#     plt.style.use('dark_background')
#     plt.title(file)
#     plt.plot(df.index, df[df.columns[0]])
#     plt.show()

df=pd.read_csv('./../data/positive_composite/NIM.csv')
# arr = ['CDR', 'CR','INF','LAS',
#        'NIA','OE','PR', 'RA',
#        'ROA','SCTA','SIZE']
arr = ['CR']
for name in arr:
    gotten=pd.read_csv('./../data/positive_composite/'+name+".csv")
    df[gotten.columns[1]]=gotten[gotten.columns[1]]
# print(df)
Y=df['NIM']
X=df[arr]
model = sm.OLS(Y, X).fit()

print(model.summary())

MissingDataError: exog contains inf or nans

In [27]:
import pandas as pd
import numpy as np

def find_nans_infs(csv_file_path):
    """
    Reads a CSV file and identifies NaNs or infinite values.

    Args:
    csv_file_path (str): Path to the CSV file.

    Returns:
    list: List of tuples containing row and column indices of NaNs or infinite values.
    """
    try:
        df = pd.read_csv(csv_file_path)
        nan_inf_indices = np.where(~np.isfinite(df))
        nan_inf_locations = list(zip(nan_inf_indices[0], nan_inf_indices[1]))
        return nan_inf_locations
    except Exception as e:
        print("An error occurred:", e)

# Example usage:
csv_file_path = './../data/positive_composite/'+'CR'+".csv"  # Provide the path to your CSV file
nan_inf_locations = find_nans_infs(csv_file_path)
if nan_inf_locations:
    print("NaNs or infinite values found at the following locations:")
    for row, col in nan_inf_locations:
        print(f"Row: {row}, Column: {col}")
else:
    print("No NaNs or infinite values found in the CSV file.")

An error occurred: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
No NaNs or infinite values found in the CSV file.


In [22]:
'./../data/positive_composite/'+'CR'+".csv"